In [53]:
sensors = []
beacons = []
with open('input.txt', 'r') as fd:
    for line in fd:
        parts = line.replace(',', '').replace('=', ' ').replace(':', '').split()
        sx, sy, bx, by = map(int, [parts[3], parts[5], parts[-3], parts[-1]])
        sensors.append((sx, sy))
        beacons.append((bx, by))

items = set(sensors + beacons)
x_vals = sorted(i[0] for i in items)
min_x, max_x = x_vals[0], x_vals[-1]
y_vals = sorted(i[1] for i in items)
min_y, max_y = y_vals[0], y_vals[-1]
# min_y = -2
dims = (min_x, min_y, max_x, max_y)

In [5]:
def visualize(sensors, beacons, dims, symbols=None):
    min_x, min_y, max_x, max_y = dims
    symbols = {} if symbols is None else symbols
    y_size = len(str(max_y)) + 1
    for r in range(min_y, max_y + 1):
        row = [f'{r: <{y_size}}']
        for c in range(min_x, max_x + 1):
            cell = (c, r)
            if cell in sensors:
                v = 'S'
            elif cell in beacons:
                v = 'B'
            elif cell in symbols:
                v = symbols[cell]
            else:
                v = '.'
            row.append(v)
        print(' '.join(row))

In [17]:
def get_covered_offsets(dist):
    covered = set()
    for x in range(dist * -1, dist + 1):
        y = dist - abs(x)
        for y1 in range(y * -1, y + 1):
            covered.add((x, y1))
    return covered

def get_covered_x_coords(coord, dist, y_val):
    covered = set()
    y_delta = abs(coord[1] - y_val)
    if y_delta > dist:
        return covered
    x_max = dist - y_delta
    return {coord[0] + x for x in range(x_max * -1, x_max + 1)}


    # x, y = coord
    # for x_offset, y_offset in get_covered_offsets(dist):
    #     if y + y_offset == y_val:
    #         covered.add((x + x_offset, y + y_offset))
    # return covered

In [21]:
seen = set()
y_val = 2000000
items_x_coords = {i[0] for i in items if i[1] == y_val}
for (sx, sy), (bx, by) in zip(sensors, beacons):
    dist = abs(bx - sx) + abs(by - sy)
    covered = set(get_covered_x_coords((sx, sy), dist, y_val)).difference(items_x_coords)
    seen.update(covered)
    # symbols = {}.fromkeys(seen, '#')
    # symbols.update({(sx, sy): 'X', (bx, by): 'O'})
    # visualize(sensors, beacons, dims, symbols)

In [22]:
len(seen)

5367037

Part 2

In [ ]:
def intersect(x_vals, left, right):
    if right <= x_vals[0][0] or left >= x_vals[-1][1]:
        return x_vals
    out = []
    for s, e in x_vals:
        if left <= s and e <= right:
            # Completely covers
            continue

        if left < s and right < s or left > e and right > e:
            out.append((s, e))
            continue

        if s < left and right < e:
            # Split
            out.append((s, left - 1))
            out.append((right + 1, e))
            continue

        # Truncate
        if left <= s and s <= right:
            s = right + 1
        elif left <= e and e <= right:
            e = left  - 1
        out.append((s, e))

    return out

In [39]:
intersect([(0, 10)], 5, 5)

[(0, 4), (6, 10)]

In [57]:
limit = 4000000
dists = [(sx, sy, abs(bx - sx) + abs(by - sy)) for (sx, sy), (bx, by) in zip(sensors, beacons)]
for row in range(limit + 1):
    p = [(0, limit)]
    for sx, sy, d in dists:
        if sy - d <= row <= sy + d:
            x_offset = d - abs(row - sy)
            left = sx - x_offset
            right = sx + x_offset
            p = intersect(p, left, right)
            if not p:
                break

    if p:
        print('found it')
        x = p[0][0]
        print(x, row, x * 4000000 + row)
        break

found it
2978645 3249288 11914583249288
